In [3]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas.io.json import json_normalize
sys.path.append('../')
from getRefXSecs import getXSecFor
import pyslha
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from collections import OrderedDict
import seaborn as sns

pd.option_context('display.max_columns', -1)

pd.options.mode.chained_assignment = None #Disable copy warnings
# plt.style.use('fivethirtyeight') #Set style
# mpl.rcParams.update({'figure.figsize' : (15,10)})  #Set general plotting options

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

In [4]:
#Merge with SModelS DataFrame
dataDF = pd.read_pickle('../EWino_scanRandom.pcl')
# print(dataDF.columns.values.tolist()) #Print all columns names

#### Set r = 0 for points with no results 

In [5]:
#Set points without results with r == 0
dataDF.fillna(value={'ExptRes.result0.r' : 0.0},inplace=True)
#Sort points according to r value:
dataDF = dataDF.sort_values(by=['ExptRes.result0.r'],ascending=False)
#Add simple flag for excluded points:
dataDF['excluded'] = dataDF['ExptRes.result0.r'] > 1.

#### Add c*tau column for charginos

In [6]:
dataDF['ctau.1000024'] = 1.967e-16/dataDF['width.1000024']

### Add ratio of total cross-section to wino cross-sections

In [7]:
# pids = [[-1000024,1000024],[1000023,1000024],[-1000024,1000023]]
# ratio = []
# for index,row in dataDF.iterrows():    
#     xsecRefSum = 0.0
#     xsecSum = 0.0
#     for pidpair in pids:
#         mass1 = row['mass.%i'%abs(pidpair[0])] 
#         mass2 = row['mass.%i'%abs(pidpair[1])] 
#         mass = (mass1+mass2)/2.
#         xsecRef = max(0,1000*getXSecFor(pidpair[0],pidpair[1],mass,13.0,'wino'))
#         xsecRefSum += xsecRef
            
#         xsecLabel = 'xsec13TeV(fb).%i_%i' %(pidpair[0],pidpair[1])
#         try:
#             xsec = row[xsecLabel]
#         except:
#             xsec = 0.0
#         xsecSum += xsec
#     if xsecRefSum > 0.0:
#         ratio.append(xsecSum/xsecRefSum)    
#     else:
#         ratio.append(0.0)

In [8]:
# dataDF['xsecRatio'] = ratio

### Get points excluded and allowed

In [9]:
excluded = dataDF[dataDF['excluded'] == True]
allowed = dataDF[dataDF['excluded'] == False]
print('Total number of points = %i' %len(dataDF))
print('Total excluded = %i'%(len(excluded)))
print('Total allowed = %i'%(len(allowed)))

Total number of points = 96629
Total excluded = 7269
Total allowed = 89360


### Create analysis label for excluded points

In [10]:
anas = excluded['ExptRes.result0.AnalysisID'].value_counts()
anas.sort_values(inplace=True,ascending=False)

    
labels = []
for index, row in excluded.iterrows():
    ana = row['ExptRes.result0.AnalysisID']
    txList = sorted(row['ExptRes.result0.TxNames'])
    label = ana + ' (' + ','.join(txList) + ')'
    label = label.replace('off','') #Group together on and off-shell topologies
    labels.append(label)
excluded['label'] = labels


In [14]:
# dataFiltered = allowed[(allowed['mass.1000024']-allowed['mass.1000022'] < 90.)
#         & (abs(allowed['mass.1000022']-150.) < 30.)
#         & (abs(allowed['mass.1000024']-175.) < 25.)
#         & (abs(allowed['mass.1000024']-allowed['mass.1000023']) < 1.0)                       
#         & (allowed['ExptRes.result0.r']/allowed['xsecRatio'] < 0.9)]
# dataFiltered[['mass.1000024','mass.1000023','mass.1000022','ExptRes.result0.r','xsecRatio','filename']]
dataFilter = excluded[(excluded['label'] == 'ATLAS-SUSY-2018-32 (TChiWW)')
                     & (excluded['mass.1000024'] < 250)]
print(len(dataFilter))
print(dataFilter[['filename','ExptRes.result0.AnalysisID','ExptRes.result0.r','ExptRes.result1.r']])

26
               filename ExptRes.result0.AnalysisID  ExptRes.result0.r  \
37251  ew_tsrtluf2.slha         ATLAS-SUSY-2018-32           1.855510   
38061  ew_tk631bjq.slha         ATLAS-SUSY-2018-32           1.730209   
18440  ew_ai_imb_5.slha         ATLAS-SUSY-2018-32           1.665554   
80691  ew_1vosje0n.slha         ATLAS-SUSY-2018-32           1.637391   
1324   ew_wo61jb12.slha         ATLAS-SUSY-2018-32           1.622996   
72080  ew_kw5e_983.slha         ATLAS-SUSY-2018-32           1.563346   
987    ew_phxcy4w3.slha         ATLAS-SUSY-2018-32           1.561608   
36118  ew_bu0w6uot.slha         ATLAS-SUSY-2018-32           1.549555   
28325  ew_n6jp0gg9.slha         ATLAS-SUSY-2018-32           1.465227   
54691  ew_pl6sgljk.slha         ATLAS-SUSY-2018-32           1.418016   
35240  ew_rupv_3k0.slha         ATLAS-SUSY-2018-32           1.388448   
9938   ew_dcazesgi.slha         ATLAS-SUSY-2018-32           1.345907   
34669  ew_s4_i2htc.slha         ATLAS-SUSY-2018-

In [ ]:
plt.figure(figsize=(14,6))
excCurves = {}
excCurves['CMS-SUS-17-004 (TChiWZ)'] =  np.genfromtxt('../EWino/data/CMS_SUS_17_004_TChiWZ_obs.csv', 
                                                      delimiter=',')
colorDict = dict(zip(excCurves.keys(),sns.color_palette("Paired",len(excCurves))))

x = dataFiltered['mass.1000024']
y = dataFiltered['mass.1000022']
# z = dataFiltered['ExptRes.result0.r']
z = dataFiltered['ExptRes.result0.r']/dataFiltered['xsecRatio']
cm = plt.cm.get_cmap('Greys')
cm = plt.cm.get_cmap('PiYG', 11) 

plt.scatter(x,y,c=z,s=20,cmap=cm,vmin=z.min(),vmax=z.max())

plt.plot([90,500],[0,410],linestyle='--',c='black')


for ana in excCurves:
    curve = excCurves[ana]
    color = colorDict[ana]
    plt.plot(curve[:,0],curve[:,1],linewidth=5,linestyle='-',color=color,label=ana)
        

lgd = plt.legend(loc=(1.3,0.2), framealpha=1.,fontsize=12)

plt.ylim(100,200)
plt.xlim(150,250)
plt.xlabel(r'$\mathregular{m_{\tilde{\chi}_1^\pm}}$ (GeV)',fontsize = 25)
plt.ylabel(r'$\mathregular{m_{\tilde{\chi}_1^0}}$ (GeV)',fontsize = 25)
plt.title("Points allowed by LHC(%i points)" %len(x))
plt.colorbar(label = r'$\sigma_{EW}/\sigma_{EW}^{wino}$')
plt.tight_layout()
plt.show()

In [ ]:
dataFiltered['ExptRes.result0.r']/dataFiltered['xsecRatio']

In [ ]:
row = allowed.loc[8885]

In [ ]:
row['mass.1000024'],row['mass.1000022'],row['filename']

In [ ]:
print(row['ExptRes.result0.r'],row['ExptRes.result0.AnalysisID'])

In [ ]:
print(row['xsec13TeV(fb).1000023_1000024'],row['xsec13TeV(fb).-1000024_1000023'])
print('Total = ',row['xsec13TeV(fb).1000023_1000024']+row['xsec13TeV(fb).-1000024_1000023'])

In [ ]:
#Cross-sections from Prospino:
# (1000023_1000024): LO = 5.64, NLO = 6.33
# (-1000024_1000023): LO = 1.88, NLO = 2.27
# Total: LO = 7.52, NLO = 8.6 (K = 1.14)

#Cross-sections from Pythia6:
# (1000023_1000024): LO = 5.958
# (-1000024_1000023): LO = 1.962
# Total: LO = 7.91

#Cross-sections for winos (LHC SUSY Xsec group):
# (1000023_1000024): NLO+NLL = 6.99
# (-1000024_1000023): NLO+NLL = 2.52
# Total: NLO+NLL = 9.51


